<a href="https://colab.research.google.com/github/akash-kaul/ecosys/blob/master/WriteQueryDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from ipywidgets import interact, interactive, fixed, interact_manual, widgets, Layout
from IPython.display import display, HTML
from IPython.utils import io
from tqdm import tqdm
from ipywidgets.widgets import VBox, HBox, Box, Label
import time
from glob import glob
import requests
from functools import wraps
import re
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))
out = widgets.Output()
form_item_layout = Layout(
    display='inline-flex',
    flex_flow='row',
    justify_content='space-between'
)
hostname = widgets.Text(
    value='',
    placeholder='TigerGraph Hostname (ex: https://tg.i.tgcloud.io)',
    # description='Hostname',
    disabled=False
)
username = widgets.Text(
    value='tigergraph',
    placeholder='TigerGraph Username',
    # description='Username',
    disabled=False
)

password = widgets.Text(
    value='',
    placeholder='TigerGraph Password',
    # description='Password',
    disabled=False,
)

graphname = widgets.Text(
    value = 'MyGraph',
    placeholder='TigerGraph Graph Name',
    disabled=False,
)

# Create a dropdown for GSQL version
version = widgets.Dropdown(
    options=["2.6.2", "3.0.0", "3.0.5"],
    value="2.6.2",
    disabled=False,
)

cert = widgets.Checkbox(
    value=False,
    # description='Cert Needed',
    disabled=False,
    indent=False
)
connect = widgets.Button(
    description='Connect To TigerGraph',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to connect to graph',
    icon='plug', # (FontAwesome names without the `fa-` prefix)
    layout=Layout(width='100%')
)
error = widgets.Label(
    value='Something went wrong!! Make sure your server is live and your information is entered correctly',
    disabled=False,
    layout=Layout(width='50%')

)
click_cancel = widgets.Button(
    description = 'No',
    disabled=False,
    button_style='danger',
    icon='ban',
    layout=Layout(width='50%')
)
click_continue = widgets.Button(
    description = 'Yes',
    disabled=False,
    button_style='success',
    icon='arrow-circle-right',
    layout=Layout(width='50%')
)
loading = widgets.Label(
            value=f'Query installing. This may take a minute...',
            disabled=False,
            layout=Layout(width='50%')
            )
form_items = [
    Box([Label(value='Hostname'), hostname], layout=form_item_layout),
    Box([Label(value='Graph Name'), graphname], layout=form_item_layout),
    Box([Label(value='Username'), username], layout=form_item_layout),
    Box([Label(value='Password'), password], layout=form_item_layout),
    Box([Label(value='Cert Needed'), cert], layout=form_item_layout),
    Box([Label(value='Client Version'), version], layout=form_item_layout),
    Box([connect], layout=form_item_layout)
]

def connectToGraph(b):
    with tqdm(total=100, leave=False) as pbar: # Controlling output on display
        out.clear_output()
        pbar.bar_format="{percentage:3.0f}%|{bar:10}|{desc}" #
        connect.icon='spinner'
        connect.description='Loading'
        connect.button_style='warning'
        #--pyTigerGraph_BEGIN--#
        pbar.set_description_str('Installing pyTigerGraph')
        with io.capture_output() as captured:
            !pip install pytigergraph
            import pyTigerGraph as tg
        pbar.update(20)
        #--pyTigerGraph_END--#

        #--Establish_Connection_BEGIN--#
        pbar.set_description_str('Connecting to Server')
        try:
            with io.capture_output() as captured:
                graph = tg.TigerGraphConnection(host=hostname.value, username=username.value, password=password.value, graphname=graphname.value, useCert=cert.value, verison=version.value)
        except Exception as e:
            pbar.close()
            out.clear_output()
            with out:
                display(error)
                connect.icon='plug'
                connect.description='Connect to TigerGraph'
                connect.button_style='info'
            return
        pbar.update(20)
        with io.capture_output() as captured:
            API_Secret = graph.createSecret()
            API_Token = graph.getToken(API_Secret, setToken=True, lifetime=None)[0]
            pbar.update(10)
        #--Establish_Connection_END--#

        #--Installing_Methods_BEGIN--#
        pbar.set_description_str('Adding Notebook Functions')
        def get_query(x):
          text = graph.gsql(f'show query {x}')
          if 'CREATE QUERY' in text:
            q_name = text.find('CREATE QUERY')
            query_text = text[q_name:]          
          return query_text
        pbar.update(10)
        #--Installing_Methods_END-#   
        #--Grab_Queries_BEGIN-#      
        pbar.set_description_str('Fetching Graph Queries')
        with io.capture_output() as captured:  
            ls = graph.gsql('ls')
        pbar.update(10)
        query = ls.find('Queries:')
        pbar.update(10)
        queries = {}
        for line in ls[query:].splitlines():
            if '-' in line:
                start = line.find('-') + 2
                end = line.find('(')
                name = line[start:end]
                queries[name] = get_query(name)
        
        pbar.set_description_str('Information Retrieved')               
        pbar.update(10)

        #Place queries in dropdown widget
        dropdown = widgets.Dropdown(
            options=queries.keys(),
            layout=Layout(width='450px'),
            disabled=False,
            )
        pbar.update(10)
        pbar.set_description_str(' Connection sucessful')
        connect.icon='check-circle'
        connect.description='Success'
        connect.button_style='success'
        time.sleep(1)
        pbar.close()
        username.disabled=True
        password.disabled=True
        hostname.disabled=True
        cert.disabled=True
        connect.disabled=True  
        graphname.disabled=True
        version.disabled=True
    x = queries[dropdown.value]
    text = widgets.Textarea(
        value=x,
        disabled=False,
        layout=Layout(height='400px', width='450px')
        )
    confirm = widgets.Button(
            description='Install Query',
            icon='cloud-upload',
            disabled=False,
            button_style='info',
            layout=Layout(width='100%'),
            )
    form_items2 = [
        Box([Label(value='Queries'), dropdown], layout=form_item_layout),
        Box([Label(value='GSQL Script'), text], layout=form_item_layout),
        Box([confirm], layout=form_item_layout),
    ]
    form2 = Box(form_items2, layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='30%'
    ))
    out2 = widgets.Output()
    display(form2, out2)

    def on_change(change):
        out2.clear_output()
        text.value = queries[change.new]
        confirm.icon='cloud-upload'
        confirm.description='Install Query'
        confirm.button_style='info'
    def install(b):
        confirm.icon='spinner'
        confirm.description='Loading'
        confirm.button_style='warning'
        with out2:
            display(loading)
        Gsql_query = text.value + 'INSTALL QUERY ' + dropdown.value
        with io.capture_output() as captured:
            graph.gsql(f'drop query {dropdown.value}')
            graph.gsql(Gsql_query)
        confirm.icon='check-circle'
        confirm.description='Success'
        confirm.button_style='success'
        out2.clear_output()
        forming = [
            Box([Label(value='Query Successfully Installed. Would you like to install another one?')], layout=form_item_layout),
            Box([click_cancel, click_continue], layout=form_item_layout),
            ]   
        former = Box(forming, layout=Layout(
                display='flex',
                flex_flow='column',
                align_items='stretch',
                width='30%'
            ))
        with out2:
            display(former)
        track = None
        def yield_for_change(widget, attribute):
            def f(iterator):
                @wraps(iterator)
                def inner():
                    i = iterator()
                    def next_i(change):
                        try:
                            i.send(change)
                        except StopIteration as e:
                            widget.unobserve(next_i, attribute)
                    widget.on_click(next_i)
                    next(i)
                return inner
            return f
        @yield_for_change(click_cancel, 'value')
        def f():
            for i in range(1):
                x=yield
                confirm.icon='cloud-upload'
                confirm.description='Install Demo'
                confirm.button_style='info'
                text.disabled = True
                dropdown.disabled = True
                confirm.disabled = True
                out2.clear_output()
                track = False
        @yield_for_change(click_continue, 'value')
        def g():
            for i in range(1):
                x=yield
                out2.clear_output()
        f()
        g()
        if track is not None:
            return
    
    dropdown.observe(on_change, 'value')
    confirm.on_click(install)

connect.on_click(connectToGraph)
form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='30%'
))
display(form, out)

Box(children=(Box(children=(Label(value='Hostname'), Text(value='', placeholder='TigerGraph Hostname (ex: http…

Output()